In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def string_to_bin(message):
    bin_ = []
    for c in message:
        bin_.append('{0:08b}'.format(ord(c)))
    return bin_

def block_int_to_bin(block):
    arr_bin = ['{0:08b}'.format(int(e)) for e in block]
    np_bin = []
    for a in arr_bin:
        r = [int(b) for b in a]
        np_bin.append(r)
    return np.array(np_bin)

def block_bin_to_int(block):
    return block

def block_random(block, cond, value):
    while True:
        r_y = random.randint(0,64)
        r_x = random.randint(0,64)
        if block[r_y][r_x] == cond:
            block[r_y][r_x] = value
            return block



class Wulee:

    def __init__(self):
        self.source = self.msg = None
        self.key = np.array([
            [1,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,1,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,0,0,0,1,0],
            [0,1,0,1,0,1,0,0]
        ])

    def load(self, path):
        self.source = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        return self
    
    def message(self, msg):
        self.msg = msg
        return self
    
    def hide(self):
        if self.source is None:
            print('No source')
        elif self.msg is None:
            print('No message')
        else:
            hidden_word_count = 0
            words_bin = string_to_bin(self.msg)
            print(words_bin)
            for y in range(0, self.source.shape[0], 8):
                hidden_bin_count = 0
                for x in range(0, self.source.shape[1]):
                    if hidden_word_count == len(self.msg):
                        return self
                    else:
                        current_word_bin = words_bin[hidden_word_count]
                        block = self.source[y:y+8,x:x+1]
                        fi = block_int_to_bin(block)
                        gi = fi
                        # print('Block khi tách ra:\n',block,'\nBlock binary (fi):\n',fi,'\nKey K:\n',self.key)
                        # SUM(fi and key)
                        and_sum = np.bitwise_and(fi, self.key).sum()
                        # SUM(key)
                        key_sum = self.key.sum()
                        current_bit = int(current_word_bin[hidden_bin_count])
                        print('\nSUM(fi and key):',and_sum,'SUM(key):',key_sum,'bit:', current_bit)
                        if and_sum % 2 == current_bit:
                            print('NOT change fi')
                        else:
                            if and_sum == 1:
                                while True:
                                        r_y = random.randint(0,7)
                                        r_x = random.randint(0,7)
                                        if fi[r_y][r_x] == 0 and self.key[r_y][r_x] == 1:
                                            fi[r_y][r_x] = 1
                                            break
                                print('and_sum = 1')
                            elif and_sum == key_sum - 1:
                                while True:
                                        r_y = random.randint(0,7)
                                        r_x = random.randint(0,7)
                                        if fi[r_y][r_x] == 1 and self.key[r_y][r_x] == 1:
                                            fi[r_y][r_x] = 0
                                            break
                                print('and_sum == key_sum - 1')
                            else:
                                while True:
                                        r_y = random.randint(0,7)
                                        r_x = random.randint(0,7)
                                        if fi[r_y][r_x] == 1 and self.key[r_y][r_x] == 1:
                                            fi[r_y][r_x] = 0
                                            break
                                print('Left option')

                        hidden_bin_count += 1
                        
                        if hidden_bin_count == 8:

                            hidden_word_count += 1
                            hidden_bin_count = 0
                            print(f'\nHide successfully word {current_word_bin}')
                            # print(f'\nNew bin block fi:\n',fi)
                    
                
        return self

    def retrieve(self):
        print(self.source[0:8,0:8])
        return self

    def draw(self):
        rgb = cv2.cvtColor(self.source, cv2.COLOR_BGR2RGB)
        plt.subplot(1,2,1)
        plt.imshow(rgb)




In [2]:
wulee = Wulee()
wulee.load('./cat.jpeg').message('d').retrieve()

[[143 145 147 146 147 147 148 148]
 [143 144 145 145 144 145 145 146]
 [144 145 146 145 144 144 146 146]
 [144 144 143 145 145 146 144 146]
 [144 143 145 145 145 144 145 145]
 [146 145 144 144 145 148 147 146]
 [147 146 145 144 145 146 148 147]
 [146 146 143 145 145 146 147 147]]
